In [6]:
from ergochemics.similarity import MorganFingerprinter, MolFeaturizer
from ergochemics.mapping import get_reaction_center
from rdkit import Chem
from ergochemics.draw import draw_reaction, draw_molecule
from IPython.display import SVG
import numpy as np
import polars as pl

def get_lhs_block_rc(am_smarts: str) -> list[int]:
    return get_reaction_center(am_smarts, mode="combined")[0]

In [7]:
mfper = MorganFingerprinter(radius=1, length=2048, mol_featurizer=MolFeaturizer())

In [8]:
prs = pl.read_parquet(
    "/home/stef/quest_data/bottle/results/test/predicted_reactions.parquet"
)
prs.head()

FileNotFoundError: No such file or directory (os error 2): /home/stef/quest_data/bottle/results/test/predicted_reactions.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [9]:
paths = pl.read_parquet(
    "/home/stef/quest_data/bottle/results/test/found_paths.parquet"
)
paths.head()

FileNotFoundError: No such file or directory (os error 2): /home/stef/quest_data/bottle/results/test/found_paths.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [ ]:
SVG(draw_reaction(prs["am_smarts"][0]))

In [ ]:
pr_smarts = prs["smarts"][0]
pr_am_smarts = prs["am_smarts"][0]
pr_rc = get_reaction_center(pr_am_smarts, mode="combined")[0]
pr_block_mol = Chem.MolFromSmiles(pr_smarts.split(">>")[0])
print(pr_smarts)
display(SVG(draw_reaction(pr_smarts)))
display(SVG(draw_molecule(pr_block_mol, highlight_atoms=pr_rc, size=(400, 300))))

In [ ]:
fp = mfper.fingerprint(pr_block_mol, reaction_center=pr_rc, rc_dist_ub=0).reshape(1, -1)
fp_dup = mfper.fingerprint(pr_block_mol, reaction_center=pr_rc, rc_dist_ub=0).reshape(-1, 1)
np.matmul(fp, fp_dup) / (np.linalg.norm(fp, axis=1).reshape(-1, 1) * np.linalg.norm(fp_dup))

In [ ]:
df = pl.read_parquet(
       "/home/stef/bottle/artifacts/rxn_x_rule_mapping/sprhea_v3_ns_x_rc_plus_4_rules.parquet"
    )
df = df.filter(
    pl.col("rxn_id").is_in(prs["analogue_ids"][0])
).with_columns(
    reaction_center=pl.col("am_smarts").map_elements(
        get_lhs_block_rc, return_dtype=pl.List(pl.Int64)
    )
)
df.head()

In [ ]:
for sma in df["smarts"]:
    print(sma)
    display(SVG(draw_reaction(sma)))

In [ ]:
kr = df.filter(
    pl.col("rxn_id") == "10729"
)
kr_smarts = kr["smarts"][0]
kr_am_smarts = kr["am_smarts"][0]
kr_rc = get_reaction_center(kr_am_smarts, mode="combined")[0]
kr_block_mol = Chem.MolFromSmiles(kr_smarts.split(">>")[0])
print(kr_smarts)
display(SVG(draw_reaction(kr_smarts)))
display(SVG(draw_molecule(kr_block_mol, highlight_atoms=kr_rc, size=(400, 300))))

In [ ]:
fp = mfper.fingerprint(pr_block_mol, reaction_center=pr_rc, rc_dist_ub=0).reshape(1, -1)
kfp = mfper.fingerprint(kr_block_mol, reaction_center=kr_rc, rc_dist_ub=0).reshape(-1, 1)
np.matmul(fp, kfp) / (np.linalg.norm(fp, axis=1).reshape(-1, 1) * np.linalg.norm(kfp))